In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'nvidia-kernel-common-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-325-updates' for glob 'nvidia*'
Note, selecting 'nvidia-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-binary' for glob 'nvidia*'
Note, selecting 'nvidia-331-dev' for glob 'nvidia*'
Note, selecting 'nvidia-304-updates-dev' for glob 'nvidia*'
Note, selecting 'nvidia-compute-utils-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-384-dev' for glob 'nvidia*'
Note, selecting 'nvidia-libopencl1-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-340-updates-uvm' for glob 'nvidia*'
Note, selecting 'nvidia-dkms-450-server' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-source-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-331-updates-uvm' for glob 'nvidi

In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

--2021-04-03 13:16:10--  https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Resolving developer.nvidia.com (developer.nvidia.com)... 152.199.0.24
Connecting to developer.nvidia.com (developer.nvidia.com)|152.199.0.24|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 [following]
--2021-04-03 13:16:10--  https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Reusing existing connection to developer.nvidia.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb?R9YgD2vUHg0XlEFPqupYB7JPBz6h5Wayc3-9OyOEo3JPEKXB2XaNQqHfuHhdMfuw5M1pfSBGHfndKlBlRoverkKwFeo0_yVLvqXiZ83ednfi7Mkwc

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Wed_Apr_11_23:16:29_CDT_2018
Cuda compilation tools, release 9.2, V9.2.88


In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-w9cb3tk6
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-w9cb3tk6
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp37-none-any.whl size=4307 sha256=2b1e3a768aad4958c75285ccbd7a8ded99697aeef2e74b32f5655d9a837bd807
  Stored in directory: /tmp/pip-ephem-wheel-cache-8xjh4h5q/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%cu
#include <curand_kernel.h>
#include <stdio.h>
#include <time.h>
#include <string.h>
#include <math.h>
#include <stdlib.h>

#include <chrono>

#define LLI_MAX 0x7fffffffffffffff
#define INT_MAX 0x7fffffff

#define K 3
#define N 10
#define ERDOS_RENYI_PROBA 0.5
#define ERDOS_RENYI_MAX_WEIGHT 50
#define PRINT_STEPS true

__global__ void create_i_centers_mat(unsigned long int seed, bool *i_centers_mat, double proba, int start, int k)
{
    int tid = threadIdx.x;

    curandState state;
    curand_init(seed, tid, 0, &state);
 
    i_centers_mat[(tid + start) * (k+1) + 0] = 1;
    bool dropped = false;
    for (int a = 1; a < k; a++)
    {
        double ranv = curand_uniform(&state);
        i_centers_mat[(tid + start) * (k+1) + a] = !dropped && (ranv <= proba);
        if (!i_centers_mat[(tid + start) * (k+1) + a])
        {
            dropped = true;
        }
    }
    i_centers_mat[(tid + start) * (k+1) + k] = 0;
}

__global__ void create_i_centers_min_dist_vert__bunches(int *i_centers_min_dist_vert, int start, int k, int n,
                                                        long long int *dist_mat, int *i_centers, int *i_centers_len,
                                                        int *i_centers_diff, int *i_centers_diff_len, int *bunches)
{
    int tid = threadIdx.x;
    int vertex = tid + start;
    
    for (int a = k; a > 0; a--)
    {
        //Possible Optimization - Load entire row and then work on it
        long long int min_distance = INT_MAX;
        for (int b = 0; b < i_centers_len[a]; b++)
        {
            if (dist_mat[n*vertex + i_centers[n*a + b]] < min_distance)
            {
                min_distance = dist_mat[n*vertex + i_centers[n*a + b]];
                i_centers_min_dist_vert[n*a + vertex] = i_centers[n*a + b];
            }
        }

        for (int b = 0; b < i_centers_diff_len[a-1]; b++)
        {
            bunches[n*vertex + i_centers_diff[n*(a-1) + b]] = 0;
            if (dist_mat[n*vertex + i_centers_diff[n*(a-1) + b]] < min_distance)
            {
                bunches[n*vertex + i_centers_diff[n*(a-1) + b]] = 1;
            }
        }
    }
    i_centers_min_dist_vert[n*0 + vertex] = vertex;
}

__global__ void SSSP_Kernel_1(int n, int start, bool *mask, long long int *cost,
                              long long int *updated_cost, int *adj_list, int *adj_list_len, long long int *weight, int *count)
{
	int tid = threadIdx.x;
	int vertex = start + tid;

	if (mask[vertex])
	{
		mask[vertex] = false;
		(*count)--;

		for (int a = 0; a < adj_list_len[vertex]; a++)
		{
			int nbr = adj_list[n * vertex + a];

			atomicMin((unsigned int *) &updated_cost[nbr], (unsigned int) cost[vertex] + (unsigned int) weight[n * vertex + nbr]);
		}
	}
}

__global__ void SSSP_Kernel_2(int n, int start, bool *mask, long long int *cost,
                              long long int *updated_cost, int *adj_list, int *adj_list_len, long long int *weight, int *count)
{
	int tid = threadIdx.x;
	int vertex = start + tid;

	if (cost[vertex] > updated_cost[vertex])
	{
		cost[vertex] = updated_cost[vertex];
		mask[vertex] = true;
		(*count)++;
	}

	updated_cost[vertex] = cost[vertex];
}

void SSSP(int source, int n, bool *mask, long long int *cost, long long int *updated_cost, bool *mask_dev, long long int *cost_dev,
            long long int *updated_cost_dev, int *adj_list_dev, int *adj_list_len_dev, long long int *weight_dev, int *count_dev)
{
    for (int a = 0; a < n; a++)
    {
        mask[a] = false;
        cost[a] = INT_MAX;
        updated_cost[a] = INT_MAX;
    }
    int count = 1;
    
    mask[source] = true;
	cost[source] = 0;
	updated_cost[source] = 0;
 
    cudaMemcpy(mask_dev, mask, n * sizeof(bool), cudaMemcpyHostToDevice);
	cudaMemcpy(cost_dev, cost, n * sizeof(long long int), cudaMemcpyHostToDevice);
	cudaMemcpy(updated_cost_dev, updated_cost, n * sizeof(long long int), cudaMemcpyHostToDevice);
    cudaMemcpy(count_dev, &count, sizeof(int), cudaMemcpyHostToDevice);
  
    while (count != 0)
	{
        SSSP_Kernel_1<<<1,n>>>(n, 0, mask_dev, cost_dev, updated_cost_dev, adj_list_dev, adj_list_len_dev, weight_dev, count_dev);  
        SSSP_Kernel_2<<<1,n>>>(n, 0, mask_dev, cost_dev, updated_cost_dev, adj_list_dev, adj_list_len_dev, weight_dev, count_dev);
        cudaDeviceSynchronize();
  
		cudaMemcpy(&count, count_dev, sizeof(int), cudaMemcpyDeviceToHost);
	}
}

void APSP(int n, long long int dist_mat[N][N], int *adj_list_dev, int *adj_list_len_dev, long long int *weight_dev)
{
    bool *mask = new bool[n];
	long long int *cost = new long long int[n];
	long long int *updated_cost = new long long int[n];
  
    bool *mask_dev;
    cudaMalloc(&mask_dev, n * sizeof(bool));
 
	long long int *cost_dev;
    cudaMalloc(&cost_dev, n * sizeof(long long int));
 
	long long int *updated_cost_dev;
    cudaMalloc(&updated_cost_dev, n * sizeof(long long int));
    
    int *count_dev;
    cudaMalloc(&count_dev, sizeof(int));
 	 
    for (int a = 0; a < n; a++)
    {
        SSSP(a, n, mask, cost, updated_cost, mask_dev, cost_dev, updated_cost_dev, adj_list_dev, adj_list_len_dev, weight_dev, count_dev);
        cudaMemcpy(cost, cost_dev, n * sizeof(long long int), cudaMemcpyDeviceToHost);

        memcpy(dist_mat[a], cost, n * sizeof(long long int));
    }
 
    delete []mask;
    delete []cost;
    delete []updated_cost;
 
    cudaFree(mask_dev);
    cudaFree(cost_dev);
    cudaFree(updated_cost_dev);
    cudaFree(count_dev);
}

long long int get_approximate_distance(int u, int v, int bunches[N][N], int i_centers_min_dist_vert[K][N], long long int dist_mat[N][N])
{
    int w = u, i = 0, tmp;
    int tmp_u = u, tmp_v = v;
 
    while (bunches[tmp_v][w] != 1)
    {
        i++;

        tmp = tmp_u;
        tmp_u = tmp_v;
        tmp_v = tmp;

        if (i >= K)
        {
            printf("ERROR! i = %d >= K = %d for u = %d and v = %d\n", i, K, u, v);
            break;
        }
        w = i_centers_min_dist_vert[i][tmp_u];
    }
 
    return dist_mat[w][tmp_u] + dist_mat[w][tmp_v];
}

int main()
{
    srand(1234);
 
    int adj_list[N][N], adj_list_len[N];
    memset(adj_list, 0, N * N * sizeof(int));
    memset(adj_list_len, 0, N * sizeof(int));
    long long int weight[N][N];
 
    for (int a = 0; a < N; a++)
    {
        weight[a][a] = 0;

        for (int b = a+1; b < N; b++)
        {
            double ranv = ((double) rand()) / RAND_MAX;
            if (ranv <= ERDOS_RENYI_PROBA)
            {
                adj_list[a][adj_list_len[a]++] = b;
                adj_list[b][adj_list_len[b]++] = a;

                weight[a][b] = 1 + (rand() % ERDOS_RENYI_MAX_WEIGHT);
                weight[b][a] = weight[a][b];
            }
            else
            {
                weight[a][b] = INT_MAX;
                weight[b][a] = INT_MAX;
            }
        }
    }
  
    if (PRINT_STEPS)
    {
        printf("Randomly Generated Graph:\n");
        for (int a = 0; a < N; a++)
        {
            printf("V%d: ", a);
            for (int b = 0; b < adj_list_len[a]; b++)
            {
                printf("(V%d, %lld) ", adj_list[a][b], weight[a][adj_list[a][b]]);
            }
            printf("\n");
        }
        printf("\n");
    }
 
    int *adj_list_dev;
    cudaMalloc(&adj_list_dev, N * N * sizeof(int));
    cudaMemcpy(adj_list_dev, &adj_list, N * N * sizeof(int), cudaMemcpyHostToDevice);
 
	int *adj_list_len_dev;
    cudaMalloc(&adj_list_len_dev, N * sizeof(int));
    cudaMemcpy(adj_list_len_dev, &adj_list_len, N * sizeof(int), cudaMemcpyHostToDevice);
 
	long long int *weight_dev;
    cudaMalloc(&weight_dev, N * N * sizeof(long long int));
    cudaMemcpy(weight_dev, &weight, N * N * sizeof(long long int), cudaMemcpyHostToDevice);

    long long int dist_mat[N][N];
    APSP(N, dist_mat, adj_list_dev, adj_list_len_dev, weight_dev);
 
    cudaFree(adj_list_dev);
    cudaFree(adj_list_len_dev);
    cudaFree(weight_dev);
 
    if (PRINT_STEPS)
    {
        printf("Distance Matrix:\n");
        for (int a = 0; a < N; a++)
        {
            printf("\tV%d", a);
        }
        printf("\n");
        for (int a = 0; a < N; a++)
        {
            printf("V%d\t", a);
            for (int b = 0; b < N; b++)
            {
                printf("%lld\t", dist_mat[a][b]);
            }
            printf("\n");
        }
        printf("\n");
    }

    auto start = std::chrono::high_resolution_clock::now();
 
    bool i_centers_mat[N][K+1], *i_centers_mat_dev;
    cudaMalloc(&i_centers_mat_dev, N*(K+1)*sizeof(bool));
 
    double proba = pow(N, -1.0/K);
    create_i_centers_mat<<<1, N>>>(time(0), i_centers_mat_dev, proba, 0, K);
    cudaDeviceSynchronize();
 
    cudaMemcpy(&i_centers_mat, i_centers_mat_dev, N*(K+1)*sizeof(bool), cudaMemcpyDeviceToHost);
    cudaFree(i_centers_mat_dev);
 
    if (PRINT_STEPS)
    {
        for (int a = 0; a < N; a++)
        {
            printf("Vertex %d\t: ", a);
            for (int b = 0; b <= K; b++)
            {
                printf("%d ", i_centers_mat[a][b]);
            }
            printf("\n");
        }
        printf("\n");
    }
  
    int i_centers[K+1][N], i_centers_len[K+1];
    memset(&i_centers, 0, sizeof(i_centers));
    memset(&i_centers_len, 0, sizeof(i_centers_len));
 
    int i_centers_diff[K][N], i_centers_diff_len[K];
    memset(&i_centers_diff, 0, sizeof(i_centers_diff));
    memset(&i_centers_diff_len, 0, sizeof(i_centers_diff_len));

    for(int a = 0; a < N; a++)
    {
        for (int b = 0; b <= K; b++)
        {
            if (i_centers_mat[a][b])
            {
                i_centers[b][i_centers_len[b]++] = a;
            }
            else
            {
                i_centers_diff[b-1][i_centers_diff_len[b-1]++] = a;
                break;
            }
        }
    }

    int *i_centers_dev, *i_centers_len_dev;
    cudaMalloc(&i_centers_dev, (K+1)*N*sizeof(int));
    cudaMemcpy(i_centers_dev, &i_centers, (K+1)*N*sizeof(int), cudaMemcpyHostToDevice);
    cudaMalloc(&i_centers_len_dev, (K+1)*sizeof(int));
    cudaMemcpy(i_centers_len_dev, &i_centers_len, (K+1)*sizeof(int), cudaMemcpyHostToDevice);
 
    int *i_centers_diff_dev, *i_centers_diff_len_dev;
    cudaMalloc(&i_centers_diff_dev, K*N*sizeof(int));
    cudaMemcpy(i_centers_diff_dev, &i_centers_diff, K*N*sizeof(int), cudaMemcpyHostToDevice);
    cudaMalloc(&i_centers_diff_len_dev, K*sizeof(int));
    cudaMemcpy(i_centers_diff_len_dev, &i_centers_diff_len, K*sizeof(int), cudaMemcpyHostToDevice);

    if (PRINT_STEPS)
    {
        for (int a = 0; a <= K; a++)
        {
            printf("I Center %d\t: ", a);
            for (int b = 0; b < i_centers_len[a]; b++)
            {
                printf("%d ", i_centers[a][b]);
            }
            printf("\n");
        }
        printf("\n");
        for (int a = 0; a < K; a++)
        {
            printf("I Center Diff %d - %d\t: ", a, a+1);
            for (int b = 0; b < i_centers_diff_len[a]; b++)
            {
                printf("%d ", i_centers_diff[a][b]);
            }
            printf("\n");
        }
        printf("\n");
    }
 
    long long int *dist_mat_dev;
    cudaMalloc(&dist_mat_dev, N*N*sizeof(long long int));
    cudaMemcpy(dist_mat_dev, &dist_mat, N*N*sizeof(long long int), cudaMemcpyHostToDevice);
 
    int i_centers_min_dist_vert[K][N], *i_centers_min_dist_vert_dev;
    cudaMalloc(&i_centers_min_dist_vert_dev, K*N*sizeof(int));
   
    int bunches[N][N], *bunches_dev;
    cudaMalloc(&bunches_dev, N*N*sizeof(int));

    create_i_centers_min_dist_vert__bunches<<<1,N>>>(i_centers_min_dist_vert_dev, 0, K, N, dist_mat_dev,
                                                     i_centers_dev, i_centers_len_dev, i_centers_diff_dev,
                                                     i_centers_diff_len_dev, bunches_dev);
    cudaDeviceSynchronize();
 
    auto stop = std::chrono::high_resolution_clock::now();
 
    cudaFree(dist_mat_dev);
    cudaFree(i_centers_dev);
    cudaFree(i_centers_len_dev);
    cudaFree(i_centers_diff_dev);
    cudaFree(i_centers_diff_len_dev);
 
    cudaMemcpy(&i_centers_min_dist_vert, i_centers_min_dist_vert_dev, K*N*sizeof(int), cudaMemcpyDeviceToHost);
    cudaFree(i_centers_min_dist_vert_dev);
    
    cudaMemcpy(&bunches, bunches_dev, N*N*sizeof(int), cudaMemcpyDeviceToHost);
    cudaFree(bunches_dev);

    if (PRINT_STEPS)
    {
        for (int a = 0; a < N; a++)
        {
            printf("Vertex %d\t: ", a);
            for (int b = 0; b < K; b++)
            {
                printf("%d ", i_centers_min_dist_vert[b][a]);
            }
            printf("\n");
        }
        printf("\n");
        for (int a = 0; a < N; a++)
        {
            printf("Vertex %d (Bunches)\t: ", a);
            for (int b = 0; b < N; b++)
            {
                if (bunches[a][b])
                {
                    printf("%d ", b);
                }
            }
            printf("\n");
        }
        printf("\n");
     
        for (int a = 0; a < N; a++)
        {
            printf("0 to %d: CALC = %d AND ACT = %d\n", a, get_approximate_distance(0, a, bunches, i_centers_min_dist_vert, dist_mat), dist_mat[0][a]);
        }
    }

    auto duration = std::chrono::duration_cast<std::chrono::microseconds>(stop - start);
    printf("\nTime Taken For Thorup-Zwick Algorithm: %lld microseconds\n", duration);
}

Randomly Generated Graph:
V0: (V1, 40) (V2, 18) (V4, 6) (V5, 23) (V7, 42) (V8, 34) 
V1: (V0, 40) (V2, 9) (V5, 31) (V6, 29) (V8, 11) 
V2: (V0, 18) (V1, 9) (V5, 13) (V7, 20) (V8, 38) 
V3: (V5, 35) (V7, 27) (V8, 45) 
V4: (V0, 6) (V5, 3) (V6, 37) 
V5: (V0, 23) (V1, 31) (V2, 13) (V3, 35) (V4, 3) (V8, 47) 
V6: (V1, 29) (V4, 37) (V7, 34) 
V7: (V0, 42) (V2, 20) (V3, 27) (V6, 34) (V9, 36) 
V8: (V0, 34) (V1, 11) (V2, 38) (V3, 45) (V5, 47) (V9, 33) 
V9: (V7, 36) (V8, 33) 

Distance Matrix:
	V0	V1	V2	V3	V4	V5	V6	V7	V8	V9
V0	0	27	18	44	6	9	43	38	34	67	
V1	27	0	9	56	25	22	29	29	11	44	
V2	18	9	0	47	16	13	38	20	20	53	
V3	44	56	47	0	38	35	61	27	45	63	
V4	6	25	16	38	0	3	37	36	36	69	
V5	9	22	13	35	3	0	40	33	33	66	
V6	43	29	38	61	37	40	0	34	40	70	
V7	38	29	20	27	36	33	34	0	40	36	
V8	34	11	20	45	36	33	40	40	0	33	
V9	67	44	53	63	69	66	70	36	33	0	

Vertex 0	: 1 1 0 0 
Vertex 1	: 1 1 1 0 
Vertex 2	: 1 0 0 0 
Vertex 3	: 1 1 0 0 
Vertex 4	: 1 1 0 0 
Vertex 5	: 1 0 0 0 
Vertex 6	: 1 1 0 0 
Vertex 7	: 1 0 0 0 
Ve